# U.S. Medical Insurance Costs

Goal: The goal of this project is to optimize the insurance cost in my future. This project will help me decide where to live if I want to have children and what my bmi should be (regarding bmi I will try to inspect if lower means alwas better). 

Data: For this project I am assuming that insurance is provided by one company. Also In the main part I won't focus on sex and smoking since I want change my sex and I will never smoke. So I will filter them out

## Preparing Data
Firstly I have imported pandas library and insurance.csv

In [2]:
import csv
import pandas as pd

starting_df = pd.read_csv("insurance.csv")
test = 1111



Now I want to check if there are any rows with Null or other "bad" values.

In [3]:

def check_for_nulls (df):
    df["nulls"] = df.apply(lambda row: row.isnull().any(), axis=1)
    return df
test_null = check_for_nulls(starting_df)

print(test_null["nulls"].unique())



[False]


In [4]:
starting_df.drop(columns=['nulls'], inplace=True)
print(starting_df.columns)

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges'], dtype='object')


## Minimizing My Insurance Cost: 

In [5]:
#Jakub's Code

## Evaluating Cost-BMI relationship:
### Steps Brainstorm:
 - What is the average cost of insurance. the ave bmi?
### Approach Breakdown
 - Add column that divides bmi into medical catagories: obese, etc
 - Find Mean and Medium Insurance cost within each bmi catagory
 - Loop through each peron in each bmi catagory, and find the mean and median cost paid by people of same/similar age, sex, region, children, smoking status (everything but bmi) and compare that to the mean and median cost at this bmi catagory. record the difference to see if there are any trends where people in a certain bmi catagory typically pay more or less than other of their same age,sec etc.
 

### 1. Aggregate BMI catagories
Add column with bmi catagories (source: [cdc.gov](https://www.cdc.gov/bmi/adult-calculator/bmi-categories.html))
- Underweight: Less than 18.5
- Healthy Weight: 18.5 to less than 25
- Overweight: 25 to less than 30
- Obesity: 30 or greater
    - Class 1 Obesity: 30 to less than 35
    - Class 2 Obesity: 35 to less than 40
    - Class 3 (Severe) Obesity: 40 or greater

Calculate Average and Median Insurance Costs by BMI Catagory


In [34]:
import numpy as np

#function to sort into bmi catagory
def bmi_catagory_sorter(bmi):
    if bmi >= 40:
        return 'f_class_3_obese'
    elif  bmi >= 35:
        return 'e_class_2_obese'
    elif  bmi >= 30:
        return 'd_class_1_obese'
    elif  bmi >= 25:
        return 'c_overweight'
    elif  bmi >= 18.5:
        return 'b_healthy_weight'
    elif  bmi < 18.5:
        return 'a_underweight'
    else:
        return 'err: input should be a number'

#Add columns with Adult BMI Category
bmi_analysis_df = starting_df
bmi_analysis_df['bmi_catagory'] = starting_df.bmi.apply(bmi_catagory_sorter)
print(bmi_analysis_df.head())

aggregate_by_bmi_catagory = bmi_analysis_df.groupby('bmi_catagory').agg({'charges':['mean','count']})
print(aggregate_by_bmi_catagory)

# avg_cost_by_bmi_catagory = bmi_analysis_df.groupby('bmi_catagory').charges.mean().reset_index().rename(columns={'charges':'bmi_cat_mean'})
# #count_by_bmi_catagory = bmi_analysis_df.groupby('bmi_catagory').charges.count().reset_index().rename(columns={'charges':'bmi_cat_count'})
# print(bmi_catagory_aggregated.sort_values(by=['bmi_cat_mean'],ascending=False))

   age     sex     bmi  children smoker     region      charges  \
0   19  female  27.900         0    yes  southwest  16884.92400   
1   18    male  33.770         1     no  southeast   1725.55230   
2   28    male  33.000         3     no  southeast   4449.46200   
3   33    male  22.705         0     no  northwest  21984.47061   
4   32    male  28.880         0     no  northwest   3866.85520   

       bmi_catagory  
0      c_overweight  
1   d_class_1_obese  
2   d_class_1_obese  
3  b_healthy_weight  
4      c_overweight  
                       charges      
                          mean count
bmi_catagory                        
a_underweight      8852.200585    20
b_healthy_weight  10409.337709   225
c_overweight      10987.509891   386
d_class_1_obese   14419.674970   391
e_class_2_obese   17022.258883   225
f_class_3_obese   16784.615546    91


### 2. Compare patience's insurance cost in each BMI catagory to the average people of similar age, region, sex, number of children and smoking status
For each BMI Catagory:
- Loop through all patiences
    - Save their insurance cost
    - Then get count and average insurance cost of all people who match this patiences age, region, sex, number of children and smoking status
    - Get difference between this person's cost and average and save to series
- Calculate averge cost difference for each BMI Catagory

In [ ]:
#functions
#This function gets the count and average of insurance cost of each patiene with the imput age +/- 2 years, sex, children, smoker, region
# - Essentially the average cost of a simlar people across all bmis and count to record the sample size
def get_avg_with_bmi_variable(ind_age,ind_sex,ind_children,ind_smoker,ind_region):
    return bmi_analysis_df[((bmi_analysis_df.age <= ind_age + 2) | (bmi_analysis_df.age >= ind_age - 2) )
                            & (bmi_analysis_df.sex == ind_sex)
                            & (bmi_analysis_df.children == ind_children) 
                            & (bmi_analysis_df.smoker == ind_smoker) 
                            & (bmi_analysis_df.region == ind_region)].agg({'charges': ['mean', 'count']})
#This function will run on all rows of bmi_analysis_df grouped by bmi_catagory 
# finding the difference between that rows charges and the average calculated by the previous function
def get_percent_dif_from_avg(row):
    i_charges = row['charges']
    i_age = row['age']
    i_sex = row['sex']
    i_children = row['children']
    i_smoker = row['smoker']
    i_region = row['region']

#get_percent_dif_from_avg = lambda row : 100*((row['charges'] - get_avg_with_bmi_variable(row['age'],row['sex'],row['children'],row['smoker'],row['region']).iloc[0].charges)/row[get_avg_with_bmi_variable(row['age'],row['sex'],row['children'],row['smoker'],row['region']).iloc[0].charges])
#get_percent_dif_from_avg_smpl_size = lambda row : row['charges'] - get_avg_with_bmi_variable(row['age'],row['sex'],row['children'],row['smoker'],row['region']).iloc[1].charges

#test functions
test = get_avg_with_bmi_variable(28,'male',0,'no','northeast')
print(test)
print(type(test))
print(test.iloc[1].charges)


           charges
mean   7959.294951
count    56.000000
<class 'pandas.core.frame.DataFrame'>
56.0


In [ ]:

bmi_analysis_df['cost_diff_from_avg'] = bmi_analysis_df.apply(get_percent_dif_from_avg,axis=1)
bmi_analysis_df['cost_diff_from_avg_smpl_size'] = bmi_analysis_df.apply(get_percent_dif_from_avg_smpl_size,axis=1)
print(bmi_analysis_df.head())

#cost_diff_by_bmi_catagory = bmi_analysis_df.groupby('bmi_catagory').cost_diff_from_avg.mean()

print(cost_diff_by_bmi_catagory)

KeyError: 27428.377302999997

In [ ]:
#Assess Data set as a whole

#Frequency of each bmi (rounded to 1's place) and 


count_each_bmi = starting_df.groupby(starting_df.bmi.apply(lambda bmi_float : round(bmi_float))).charges.count().reset_index().rename(columns={'charges': 'bmi-freq'})
print(bmi_cost_2)
 
avg_cost_each_bmi = starting_df.groupby(starting_df.bmi.apply(lambda bmi_float : round(bmi_float))).charges.mean().reset_index()
print(bmi_cost_3.head(10))



   index  bmi
0      0   28
1      1   34
2      2   33
3      3   23
4      4   29
5      5   26
6      6   33
7      7   28
8      8   30
9      9   26
   bmi  bmi-freq
0   16         1
1   17         9
2   18        11
3   19         8
4   20        24
5   21        23
6   22        35
7   23        50
8   24        57
9   25        62
   bmi       charges
0   16   1694.796400
1   17   6076.167378
2   18  11402.702809
3   19  10382.876080
4   20   8756.253285
5   21   7006.185911
6   22  10432.877525
7   23  10104.465317
8   24  11675.051279
9   25  12203.856403
